# 挂载google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive 
!google-drive-ocamlfuse drive

In [0]:
!pwd
import os
os.chdir('./drive/COMP5318/comp5318 assignment2/')
!pwd
!ls

/content
/content/drive/COMP5318/comp5318 assignment2
comp5318_assignment2_PCA_LR.ipynb  dataset  drive  共享


#assingment

##预处理

In [0]:
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
import scipy.io as scio
from sklearn.decomposition import PCA
import numpy as np
import time
import gc

In [0]:
#PCA part
train_path = "./dataset/train_32x32.mat"
data = scio.loadmat(train_path)

train_X = data["X"]
train_y = data["y"]
train_X = train_X.T
print(train_X.shape)
print(train_y.shape)

tmp = []
for row in train_X:
    tmp.append(row.flatten())
# release training set memory
train_X = None
gc.collect()

train_X = np.array(tmp)
tmp = None

train_X = train_X - train_X.mean(axis=0)
train_X.shape

# PCA decrease the dimension of the training set into d dimensions.
# d is a configurable variable (hyperparameter)
# In this case d = 50.
d = 50
pca = PCA(n_components=d)
start = time.time()
train_X = pca.fit(train_X).transform(train_X)
end = time.time()
print("shape: {}".format(str(train_X.shape)))
print("time consumption: {}s".format(end-start))
dataset = np.hstack((train_X,train_y))


#print(np.shape(train_y))

def train_test_split_10(train_X, train_y,k):
    num_of_examples = len(train_X)
    start_jump = (num_of_examples//10)*k
    end_jump = (num_of_examples//10)*(k+1)
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]
    for i in range(len(train_X)):
        if i<=start_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        elif i>end_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        else:
            X_test.append(train_X[i])
            y_test.append(train_y[i])

    return X_train,X_test,y_train,y_test

X_train, X_test, y_train, y_test =  train_test_split_10(train_X, train_y,0)
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(73257, 3, 32, 32)
(73257, 1)
shape: (73257, 50)
time consumption: 33.60878396034241s
(65932, 50)
(65932, 1)
(7325, 50)
(7325, 1)


##LR

In [0]:
from sklearn.linear_model import LogisticRegression   
from sklearn.metrics import classification_report, confusion_matrix
  
y_train=np.array(y_train)
y_test = np.array(y_test)
start = time.time()
classifier = LogisticRegression()    
classifier.fit(X_train, y_train)   
end = time.time()
print("time consumption: {}s".format(end-start))
train_prediction = classifier.predict(X_train)
valid_prediction = classifier.predict(X_test)   
   
print('Training score is', classifier.score(X_train, y_train))
print('Validation score is', classifier.score(X_test, y_test))
print(type(y_train))
print(type(train_prediction))
print('Classification report of training data:\n', classification_report(y_train, train_prediction))
print('Confusion Matrix of training data:\n', confusion_matrix(y_train, train_prediction))

print('Classification report of validation data:\n', classification_report(y_test, valid_prediction))
print('Confusion Matrix of validation data:\n', confusion_matrix(y_test, valid_prediction))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


time consumption: 89.4840350151062s
Training score is 0.24149123339197962
Validation score is 0.2465529010238908
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Classification report of training data:
              precision    recall  f1-score   support

          1       0.23      0.78      0.36     12457
          2       0.25      0.34      0.29      9554
          3       0.24      0.10      0.14      7667
          4       0.32      0.13      0.18      6671
          5       0.28      0.08      0.13      6181
          6       0.28      0.04      0.07      5166
          7       0.18      0.02      0.04      5068
          8       0.20      0.01      0.02      4530
          9       0.17      0.01      0.01      4204
         10       0.23      0.08      0.12      4434

avg / total       0.24      0.24      0.18     65932

Confusion Matrix of training data:
 [[9705 1772  159  326  121   55   53   20    7  239]
 [5225 3292  359  259   81   66   61   15   26  170]
 [4974 1223  781 